In [1]:
import os
import nltk
import string
import re
import numpy as np
import pandas as pd
from numpy import array
from numpy import argmax
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize import RegexpTokenizer
from sklearn.decomposition import PCA

In [2]:
ngram_range_inp=(1,2)
filename = "D:/Pepsico/input_data_1.xlsx"

# Text Normalization

In [3]:
exclude = set(string.punctuation) 
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')
newStopWords = ['from']
stop_words.extend(newStopWords)
table = str.maketrans('', '', string.punctuation)

porter = PorterStemmer()

def normalize_document(doc):
    # tokenize document
    tokens = doc.split()
    # remove punctuation from each word
    tokens = [w.translate(table) for w in tokens]
    # convert to lower case
    lower_tokens = [w.lower() for w in tokens]
    #remove spaces
    stripped = [w.strip() for w in lower_tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter stopwords out of document
    filtered_tokens = [token for token in words if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [4]:
# Vectorization parameters

# Range (inclusive) of n-gram sizes for tokenizing text.
NGRAM_RANGE = ngram_range_inp

# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Whether text should be split into word or character n-grams.
# One of 'word', 'char'.
TOKEN_MODE = 'word'

# Minimum document/corpus frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY = 2

# Limit on the length of text sequences. Sequences longer than this
# will be truncated.
MAX_SEQUENCE_LENGTH = 500


def ngram_vectorize(train_texts, train_labels):
    """Vectorizes texts as ngram vectors.

    1 text = 1 tf-idf vector the length of vocabulary of uni-grams + bi-grams.

    # Arguments
        train_texts: list, training text strings.
        train_labels: np.ndarray, training labels.
        val_texts: list, validation text strings.

    # Returns
        x_train, x_val: vectorized training and validation texts
    """
    # Create keyword arguments to pass to the 'tf-idf' vectorizer.
    kwargs = {
            'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': TOKEN_MODE,  # Split text into word tokens.
            'min_df': MIN_DOCUMENT_FREQUENCY,
    }
    vectorizer = TfidfVectorizer(**kwargs)

    # Learn vocabulary from training texts and vectorize training texts.
    x_train = vectorizer.fit_transform(train_texts)

    # Vectorize validation texts.
    #x_val = vectorizer.transform(val_texts)

    # Select top 'k' of the vectorized features.
    #selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    #selector.fit(x_train, train_labels)
    #x_train = selector.transform(x_train)
    #x_val = selector.transform(x_val)

    x_train = x_train.astype('float32')
    #x_val = x_val.astype('float32')
    return x_train

In [5]:
fsha_data = pd.read_excel(filename)

In [6]:
fsha_data= fsha_data.rename(columns={' pH': 'pH'})
fsha_data= fsha_data.rename(columns={' otherFSA': 'otherFSA'})
fsha_data= fsha_data.rename(columns={' targetMarket': 'targetMarket'})
fsha_data= fsha_data.rename(columns={' newIngradient': 'newIngredient'})

# Model preparation (taking few features)

In [7]:
dataframe_model = fsha_data[['preservatives', 'pH', 'waterActivity', 'packaging', 'otherFSA', 'allergens', 
             'prod_storageDist', 'foodSafety_prodClaims', 'targetMarket', 'newIngredient']]

In [8]:
train_df = dataframe_model

In [9]:
train_df

,preservatives,pH,waterActivity,packaging,otherFSA,allergens,prod_storageDist,foodSafety_prodClaims,targetMarket,newIngredient
0,E262 Sodium acetates (i) Sodium acetate (ii) S...,NaN,low,NaN,NaN,Base allergens: None\nTop seasoning: Milk\nCro...,ambient,No preservatives,No,Yes
1,NaN,NaN,low,NaN,NaN,"Base allergens: None\nTop seasoning: Milk, Glu...",ambient,No preservatives,no,Yes
2,None,NaN,NaN,Adhesive laminate with no Nitrogen flushing,NaN,Wheat Gluten,Store in a cool dry place at ambient temperature,No claims made,"Small pieces can be a choke hazard, Not suitab...",No
3,not present in seasoning,not known yet for reformulated seasoning - exi...,"Max 5% in seasoning, FP aim 1,8% and max 2,2%",Packed in a protective atmosphere,None,"milk, Soya, Barley",none,none,no,No
4,None,n/a,"Pellet (<0.6), Finished Product (<0.15)",Nitrogen flushed?,Moisture Content of Pellet : 10 - 12 %,Inherent:\nCheddar Playz: Cereals containing G...,Ambient,n/a,"Suitable for consumers, potential choking haza...",Yes
5,None,n/a,"Pellet (<0.6), Finished Product (<0.15)",Pellet WIP packaging not N2 flushed. 6 month s...,Moisture Content of Pellet : 10 - 12 %,Pellets Inherent:\nCereals containing Gluten -...,Ambient,n/a,"Suitable for consumers, potential choking haza...",Yes
6,NaN,NaN,Low,Lays is nitrogen flushed,NaN,Standard allergen management for Seasoning. Th...,RTE,None,"Yes, no change to current product",No
7,None,Not applicable,low moisture food,no modified atmosphere,Not applicable,Allergens in seasonings:\nCQ69 Vegetable Blend...,"Store in a dry, cool and away from sun place.","claim 1 ""from corn""\nclaim 2 ""not fried""\nclai...","People with allergies to milk, lactose, peanuts.",Yes
8,E262 Sodium acetates (i) Sodium acetate (ii) S...,NaN,"low, approximately 0,04",nitrogen flushed bags,NaN,Inherent\n\nParmesan Cheese & Roasted Garlic B...,ambient,50% less fat comparing to fried potato chips,no,Yes
9,May be in topping/seasoning ingredients TBC fi...,NaN,Low water activity,Packaging conditions TBC,NaN,"Inherent - Wheat, Gluten, Sulphites, Soya, Mil...",N/A ambient,Allergen suffers. Made in a factory that handl...,Allergen suffers. Made in a factory that handl...,Yes


# One-hot encoding of categorical variables

In [10]:
def categorical_encode(data):
    #data = train_df['prod_storageDist']
    values = array(data)
    print(values)
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    print(integer_encoded)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    print(onehot_encoded)
    return onehot_encoded

In [11]:
#Input for PCA (can be taken as cmd-line parameters)
n_components = 2
whiten = False
random_state = 42
svd_solver="full"

In [22]:
#Function to pre process text data
def preprocess_text(train_df,y):
    storage_encoded = categorical_encode(train_df['prod_storageDist'])
    norm_allergens = normalize_corpus(train_df['allergens'])
    print(train_df['allergens'][0])
    print(norm_allergens[0])
    train_labels = y
    print(storage_encoded.shape)
    x_train = ngram_vectorize(norm_allergens, train_labels)
    x_train=x_train.toarray()
    print(x_train.shape)
    pca = PCA(n_components=n_components,svd_solver=svd_solver,whiten=whiten, random_state=42)
    x_pca = pca.fit_transform(x_train)
    print(x_pca.shape)
    x_train = np.concatenate((storage_encoded,x_pca),axis=1)
    return x_train

In [23]:
#Convert Yes/No labels in output to 1 or 0
train_y=[]
for i in range (len(fsha_data)):
    if fsha_data["potentialMicrobial"][i] =='Yes':
        train_y.append(1)
    else:
        train_y.append(0)

In [24]:
y = train_y

In [25]:
x_train = preprocess_text(train_df,y)

['ambient' 'ambient' 'Store in a cool dry place at ambient temperature'
 'none' 'Ambient ' 'Ambient ' 'RTE'
 'Store in a dry, cool and away from sun place.' 'ambient' 'N/A ambient']
[5 5 3 6 0 0 2 4 5 1]
[[0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0.]]
Base allergens: None
Top seasoning: Milk
Cross-contact (Suadiye): Gluten, Peanuts, Soya
Cross-contact (Tarsus): Gluten (Wheat, Barley), Peanuts, Soya
base allergens none top seasoning milk crosscontact suadiye gluten peanuts soya crosscontact tarsus gluten wheat barley peanuts soya
(10, 7)
(10, 51)
(10, 2)


C:\Users\103467\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [199]:
x_train.shape

(10, 9)

In [200]:
train_y

[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]

# Create Data Pipeline to select best features and run Classifier

In [202]:
selector = SelectKBest(f_classif, k=10)
clf = svm.SVC(kernel='linear')
selector_svm = Pipeline([('selector', selector), ('svc', clf)])

In [203]:
selector_svm.set_params(selector__k=5, svc__C=.1).fit(x_train, y)

C:\Users\103467\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\__init__.py:93: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


Pipeline(memory=None,
     steps=[('selector', SelectKBest(k=5, score_func=<function f_classif at 0x0000000011C22510>)), ('svc', SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

# Fit ML Models from a List and print / save the best model

In [164]:
names = ["Ridge Classifier", "Linear SVC"]

classifiers = [
    RidgeClassifier(tol=1e-2, solver="lsqr"),
    svm.SVC(kernel='linear')]

In [159]:
X, y = x_train,y

In [160]:
X.shape

(10, 9)

In [162]:
datasets = [(X,y)]

In [169]:
scores = []
for ds_cnt, ds in enumerate(datasets):
    # preprocess dataset, split into training and test part
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=42)
    print(X_train.shape)
    print(X_test.shape)
     # iterate over classifiers
    for name, clf in zip(names, classifiers):
        #ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        scores.append(score)
        print(score)    

(9, 9)
(1, 9)
0.0
1.0


In [171]:
index=np.argmax(scores)

In [172]:
index

1

In [173]:
print("Best classifier is:"+names[index])

Best classifier is:Linear SVC


In [179]:
def benchmark_models(clf):
    print('_' * 80)
    print("Training: ")
    #print(clf)

    clf.fit(x_train, y)

    pred = clf.predict(x_train)

    score = metrics.f1_score(y, pred)
    print("accuracy:   %0.3f" % score)

    #clf_descr = str(clf).split('(')[0]
    return  clf,score

In [180]:
scores = []
models=[]
for clf, name in (
     (RidgeClassifier(tol=1e-2, solver="lsqr"), "Ridge Classifier"),
        (svm.SVC(kernel='linear'), "Support Vector Classifier") ):
    print('=' * 80)
    print(name)
    model,score = benchmark_models(clf)
    models.append(model)
    scores.append(score)

Ridge Classifier
________________________________________________________________________________
Training: 
accuracy:   0.000
Support Vector Classifier
________________________________________________________________________________
Training: 
accuracy:   0.000


C:\Users\103467\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [181]:
scores

[0.0, 0.0]

In [182]:
models

[RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
         max_iter=None, normalize=False, random_state=None, solver='lsqr',
         tol=0.01), SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False)]

# Save the best model for future use

In [183]:
models[0]
filename = 'D:/Pepsico/finalized_model.sav'
pickle.dump(models[0], open(filename, 'wb'))

In [184]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_train, y)
print(result)

0.8
